In [18]:
from __future__ import print_function
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
import numpy as np


In [26]:
# Data paths and constants
data_path = r"roman_english.txt"  # Update with your data path
num_samples = 10000  # Update with appropriate number of samples
latent_dim = 256  # Update with desired latent dimensionality
batch_size = 64  # Update with desired batch size
epochs = 100  # Update with desired number of epochs

In [36]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

# Load Roman Urdu and English data
lines= None
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

print(lines)

['Hi\t\tJi', 'Kaise ho?\t\tHow are you?', 'Kese ho?\t\tHow are you?', 'Hello\t\tHello', 'Me theek hu\t\tI am good', 'Me theek hu\t\tI am fine', 'Mai theek hu\t\tI am good', 'Mai theek hu\t\tI am fine', 'Tum kya kar rahe ho?\t\tWhat are you doing?', 'Tum kya kar rhy ho?\t\tWhat are you doing?', 'Mera naam\t\tMy name is', 'Abhi khana khate ho?\t\tAre you eating right now?', 'Abhi khana khatay ho?\t\tAre you eating right now?', 'Mera dost acha ha\t\tMy friend is good', 'Mera dost acha nahi ha\t\tMy friend is not good', 'Mera dost acha nahi ha\t\tMy friend is bad', 'Mera dost bekaar hai\t\tMy friend is useless', 'Mera dost sachha ha\t\tMy friend is truthful', 'Ye duniya faani ha\t\tThis world is temporary', 'Ye dunya faani ha\t\tThis world is temporary', 'Ye\t\tThis', 'Kya?\t\tWhat?', 'Tumhara\t\tYours', 'Meme\t\tmeme', 'Ek meme aayi ha\t\t\ta meme has arrived', 'mere pe ek meme banao\t\tmake a meme on me', 'Kya\t\tWhat', 'Tum\t\tYou', 'Mai\t\tMe', 'Khubsurat\t\tBeautiful', 'Jab\t\tWhen', 

In [37]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
    print(target_text)

	Ji

	How are you?

	How are you?

	Hello

	I am good

	I am fine

	I am good

	I am fine

	What are you doing?

	What are you doing?

	My name is

	Are you eating right now?

	Are you eating right now?

	My friend is good

	My friend is not good

	My friend is bad

	My friend is useless

	My friend is truthful

	This world is temporary

	This world is temporary

	This

	What?

	Yours

	meme

		a meme has arrived

	make a meme on me

	What

	You

	Me

	Beautiful

	When

	When

	Book

	Book

	News

	Newspaper

	News Paper

	Truthful

	Truthful

	if

	if

	Why

	Why?

	Why?

	Time

	Do your work

	Do your own work

	Khud

	Sometimes

	Sometimes

	May I know your name?

	May I know your name?

	Ok

	Ok

	Its ok

	Approx

	Approximate

	Person

	This person

	Go away

	Go away

	Run

	Run

	Please

	Please

	Time

	No

	No

	Haan

	Han

	Jee

	Why not?

	Why not?

	Why not?

	Food

	Eat food

	Eat food

	Eat food

	I have a toy

	I have one toy

	Toy

	Play

	Will you play?

	Will you play

ValueError: not enough values to unpack (expected 2, got 1)

In [38]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [39]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 188
Number of unique input tokens: 47
Number of unique output tokens: 49
Max sequence length for inputs: 34
Max sequence length for outputs: 28


In [40]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [41]:
"""Turn the sentences into 3 Numpy arrays, encoder_input_data, decoder_input_data, decoder_target_data:

encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters) containing a one-hot vectorization of the English sentences
decoder_input_data is a 3D array of shape (num_pairs, max_french_sentence_length, num_french_characters) containing a one-hot vectorization of the French sentences
decoder_target_data is the same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :]
"""

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')


In [42]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [43]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [44]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [45]:
#Train a basic LSTM-based Seq2Seq model to predict decoder_target_data given encoder_input_data and decoder_input_data

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [46]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')


Epoch 1/100


3/3 [==============================] - 6s 679ms/step - loss: 3.6839 - accuracy: 0.3845 - val_loss: 2.0972 - val_accuracy: 0.6429
Epoch 2/100
3/3 [==============================] - 1s 161ms/step - loss: 1.9109 - accuracy: 0.6788 - val_loss: 1.7574 - val_accuracy: 0.6429
Epoch 3/100
3/3 [==============================] - 1s 168ms/step - loss: 1.5431 - accuracy: 0.6788 - val_loss: 1.6607 - val_accuracy: 0.6429
Epoch 4/100
3/3 [==============================] - 1s 183ms/step - loss: 1.4896 - accuracy: 0.6788 - val_loss: 1.6101 - val_accuracy: 0.6429
Epoch 5/100
3/3 [==============================] - 0s 159ms/step - loss: 1.4424 - accuracy: 0.6788 - val_loss: 1.5700 - val_accuracy: 0.6429
Epoch 6/100
3/3 [==============================] - 1s 155ms/step - loss: 1.4522 - accuracy: 0.6788 - val_loss: 1.9228 - val_accuracy: 0.6429
Epoch 7/100
3/3 [==============================] - 1s 157ms/step - loss: 1.5562 - accuracy: 0.6776 - val_loss: 1.6187 - val_accuracy: 0.6429
Epoch 8/10

c:\Users\AAHIL ALWANI\Desktop\FYP\Web App\Meme_AI\meme\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [47]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

In [48]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [49]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [50]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [54]:
# # Take input from user
# input_sentence = input("Enter a sentence in english: ")

# # Vectorize the input sentence
# input_seq = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
# for t, char in enumerate(input_sentence):
#     input_seq[0, t, input_token_index[char]] = 1.
# input_seq[0, t + 1:, input_token_index[' ']] = 1.

# # Decode the input sentence
# decoded_sentence = decode_sequence(input_seq)

# # Print the decoded sentence
# print('Decoded sentence:', decoded_sentence)

In [ ]:
for seq_index in range(200):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    #print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 24ms/step
-
Input sentence: Hi
Decoded sentence: Why

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Kaise ho?
Decoded sentence: Ho                           
1/1 [==============================] - 0s 12ms/step
-
Input sentence: Kese ho?
Decoded sentence: Howlllyyou

1/1 [==============================] - 0s 23ms/step
-
Input sentence: Hello
Decoded sentence: Whe

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Me theek hu
Decoded sentence: I                            
1/1 [==============================] - 0s 33ms/step
-
Input sentence: Me theek hu
Decoded sentence: I                            
1/1 [==============================] - 0s 33ms/step
-
Input sentence: Mai theek hu
Decoded sentence: I                            
1/1 [==============================] - 0s 20ms/step
-
Input sentence: Mai theek hu
Decoded sentence: I                            
1/1 [==============================] - 0s 33ms